In [1]:
import torch
from langchain_community.document_loaders import DirectoryLoader,PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer,AutoModelForCausalLM,pipeline
from langchain_community.vectorstores import FAISS

c:\KM\Virtual ENV\venv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader=DirectoryLoader('./data/',glob='./*.pdf',loader_cls=PyMuPDFLoader)
docs=loader.load()

In [3]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=750,chunk_overlap=150,separators=['\n\n','\n','.',' '])
chunks=text_splitter.split_documents(docs)
len(chunks)

470

In [4]:
embeddings=HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')
vector_db=FAISS.from_documents(chunks,embeddings)
vector_db.save_local('RAG Vector DB')

In [5]:
MODEL='Qwen/Qwen2.5-1.5B-Instruct'
tokenizer=AutoTokenizer.from_pretrained(MODEL)
model=AutoModelForCausalLM.from_pretrained(MODEL,device_map='auto',dtype=torch.bfloat16,low_cpu_mem_usage=True)
pipe=pipeline(task='text-generation',temperature=0.3,do_sample=True,tokenizer=tokenizer,model=model,max_new_tokens=512,repetition_penalty=1.1,
              no_repeat_ngram_size=3)

Device set to use cuda:0


In [6]:
from sentence_transformers import CrossEncoder
rerank_model=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device='cuda')

In [7]:
import os
def format_prompt(question):
    initial_search=vector_db.similarity_search(query=question,k=20)
    pairs=[[question,doc.page_content] for doc in initial_search]
    scores=rerank_model.predict(pairs)
    scored_results=sorted(zip(scores,initial_search),key=lambda x:x[0],reverse=True)
    search_results=[doc for score,doc in scored_results[:3]]
    context_list=[]
    for doc in search_results:
        full_path=doc.metadata.get('source','unknown')
        file_name=os.path.basename(full_path)
        page_num=doc.metadata.get('page',0) + 1
        header=f'[Doc : {file_name} | Page : {page_num}]'
        context_list.append(f"{header}\n{doc.page_content}")
    context='\n\n-\n\n'.join(context_list)

    prompt=f'''You are a financial analyst assistant. Answer the question using ONLY the provided context.

IMPORTANT RULES:
1. Be concise - maximum 500 words
2. Always cite sources: [Doc: filename | Page: X]
3. If context is insufficient, state: "Based on available documents, I cannot fully answer this."
4. No speculation beyond the documents
5. For financial metrics, copy exact numbers from source

Question: {question}

Context:
{context}

Answer (concise, cited):'''

    response=pipe(prompt,return_full_text=False)
    return response[0]['generated_text']



In [8]:
print(format_prompt('What Role does Climate Change Play in Finance and investments?'))

 Climate change plays a significant role in finance and investments through various mechanisms such as increased insurance costs, higher energy prices, and regulatory pressures. According to Carè (2003) and Wang et al. (219), climate-related financial risk has been identified by several studies including those conducted by U.S community banks, global banks, and China's banking sector. These studies suggest that climate change poses both direct and indirect threats to financial stability, influencing profitability and overall economic performance. Additionally, research like Chan et al.'s (24) highlights how green monetary and macro-prudential measures can mitigate these risks. Furthermore, the work by Kim and colleagues (25) underscores the importance of understanding climate policy uncertainty in relation to corporate environmental risks. Overall, addressing climate change requires comprehensive strategies that include not only mitigation but also adaptation efforts to ensure sustaina

In [9]:
print(format_prompt('How does Oil Prices affect the stock market?'))

 Oil prices have a significant impact on the stock markets, especially in volatile periods. Studies like Park and Rattis (2) and Bjørnlands (3) show that a one percentage point increase in oil prices can lead to a decrease in stock prices by up to two and a half percent. This effect is more pronounced in oil-importing economies where higher oil costs directly translate into increased production costs for businesses. Additionally, the dynamic nature of these relationships means that while there may be some stability over longer periods, they are not static but rather evolve based on economic conditions and policy responses. Therefore, investors should consider incorporating oil price forecasts when making investment decisions, as it can significantly influence overall market sentiment and volatility. [Source: Doc: Dynamic Oil Pressure Stock Volatilty Paper.pdf, Pages: 2-4] Based on the provided information, oil price fluctuations indeed play a crucial role in affecting the stock perform

In [10]:
print(format_prompt('Explain Carbon Mitigation Strategy'))

 The carbon mitigation strategy outlined in the document involves setting forward-looking plans for reducing carbon emissions over a period of \(N\) steps, denoted as \(\pi = (\gamma_i)_{i=0}^{N-1}\). Each \(\gamma_i\) represents an emission reduction effort at step \(i\), where \(\forall i = 1:N-1\), \(\text{where } \gamma_i \in [0, \gamma_{\text{max}}]\) indicates that each reduction can range between zero and a maximum value \(\mathbf{\gamma}_{\text {max}}\). These reductions are applied sequentially across different scenarios, with the total reduction amounting to the sum of all \(\sum_{i} \gamma_0\).

The strategy also considers the impact of these emissions reductions on both intensity and sales, which are influenced by factors such as the logarithmic moments of idiosyncratic shocks (\(\Delta \varepsilon_I^i, \Delta \varphi_S^i\)) and market sensitivities (\(IR_s^i\) and \(MS_s^j\)). The document suggests that the firm's anticipations regarding the specific transition scenario wi

In [11]:
print(format_prompt('Explain Default and how is the probability of Default is computed?'))

 The probability of a company's default can be estimated through a combination of mathematical models and simulation techniques. According to the paper referenced, the probability is calculated using a modified version of the Black-Cock model, specifically tailored for corporate credit risk analysis. This involves modeling both the firm's total assets and debt as stochastic processes based on their respective business models and economic scenarios. The key step in determining the probability involves evaluating the likelihood of non-payment over time, often referred to as the "probability of default." This assessment is done through nested Monte Carlo simulation methods, where multiple paths are simulated under different economic conditions and strategies to assess the impact of these factors on the probability. Essentially, it's a sophisticated way of predicting financial risks by considering various possible outcomes and their probabilities across different periods. [Doc : Corporate 

In [12]:
print(format_prompt('Explain the role of Paris Agreement'))

 The Paris Agreement plays a crucial role in motivating governments to implement concrete measures towards mitigating climate change. It establishes a framework where countries commit to reducing greenhouse gas emissions through Nationally Determined Contributions (NCDs), aiming to limit global temperature rise below 2 degrees Celsius above pre-industrial levels and pursue efforts to limit it to 1.5 degrees Celsius. This commitment encourages nations to develop and adhere to stringent emission reduction targets, fostering a collective effort to address climate change risks. Additionally, the Paris Accord provides a legal basis for countries to collaborate internationally, ensuring accountability and facilitating coordinated action among signatories. By setting ambitious yet achievable goals, the agreement serves as a catalyst for transformative changes in energy systems, transportation, and other sectors, ultimately contributing to a more resilient and sustainable future. [Doc :Climate